In [15]:
import pandas as pd
import requests
from cssselect import GenericTranslator
from lxml.etree import XPath # delete
from lxml import html
from bs4 import BeautifulSoup # delete
import wget
from zipfile import ZipFile
import fnmatch
import os
import random 
from time import sleep

In [69]:
pd.set_option('display.max_rows', 1000)

In [38]:
path = os.getcwd()

In [2]:
# User Agent
headers = {
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36",
}

In [3]:
# Wikipedia site with social revolts records
url = "https://en.wikipedia.org/wiki/List_of_revolutions_and_rebellions"

In [8]:
respuesta = requests.get(url, headers=headers)

In [9]:
# HTLM tree parser

parser = html.fromstring(respuesta.text)


## This is working fine.
### Here I parse the text of a given decade block from the wikipedia page using *xpath*

In [ ]:
# ingles = parser.get_element_by_id('/html/body/div[3]/div[3]/div[5]/div[1]/ul[22]')
ingles = parser.xpath('/html/body/div[3]/div[3]/div[5]/div[1]/ul[22]')
for i in ingles:
    print (i.text_content())

### This is working fine too. A different method with css selectors. Here I parse the text of a given decade block from the wikipedia page using *css selectors.*

In [ ]:
year = parser.xpath(GenericTranslator().css_to_xpath("ul:nth-child(100) li"))
for text in year:
    print(text.text_content())

## Finidng the links
### here you can find the links for a given decade

In [ ]:
links = parser.xpath(GenericTranslator().css_to_xpath('ul:nth-child(100) a'))
for link in links:
#     print(link.text_content())
    print('https://en.wikipedia.org' + link.attrib['href'])

# Use another wikipedia page source 

## I look for a more simple page to scrap the dataset


In [11]:
url_2 = 'https://en.wikipedia.org/wiki/List_of_protests_in_the_21st_century'

In [12]:
respuesta_2 = requests.get(url_2, headers=headers)

In [13]:
# HTLM tree parser

parser_2 = html.fromstring(respuesta_2.text)


In [ ]:
wiki_data = parser.xpath(GenericTranslator().css_to_xpath("ul:nth-child(100) li"))
for text in wiki_data:
    print(text.text_content())

## This is working fine.
### Here I parse the text of a given decade block from the wikipedia page using *xpath*

protest = parser_2.xpath(GenericTranslator().css_to_xpath('dt'))
for text in protest:
    print(text.text_content())
    print()

## Extract socioeconomical data from the worlbank

### Create a Function that make a call to the API and return a data frame

In [245]:
#####################

# This function make a specific dataseries request to the worlbank API and return a pandas df as output. 
# It takes 3  arguments:
# 1. Indicator Code "indic_code", 
# 2. year range "year_s", 
# 3. Path destination "destination"

#####################


def API_call_to_df(indic_code, 
                   year_s = '2000:2020', 
                   destination = path + "/" + "WB_datasets"):
    WB_url_seed = 'http://api.worldbank.org/v2/country/all/indicator/' + indic_code
    my_params = {'date' : year_s,
                'incomelevel' :'',
                'downloadformat' : 'csv',
                'per_page' : '304'}
    
    for file in os.listdir(destination):
        if fnmatch.fnmatch(file, "API_" + indic_code.upper() + "*"):
            os.remove(destination + "/" + file)
            response = requests.get(WB_url_seed, params = my_params)
            my_zip_file = wget.download(response.url, out= destination)
            print("file exists and it will be replaced")
            break
            
    else:
        response = requests.get(WB_url_seed, params = my_params)
        my_zip_file = wget.download(response.url, out= destination)
        
    with ZipFile(my_zip_file, 'r') as zipObj:
            for content in zipObj.namelist():
                if fnmatch.fnmatch(content, 'API_*'):
                    #print(content) #wihtin the content of the zip file find and extract the csv file that contain the data
                    my_filename = content
                    zipObj.extract(content, path = destination)
                    my_df = pd.read_csv(destination + '/' + my_filename, 
                                                header = 2,
                                                usecols = [*range(0, 25)])
                    os.remove(destination + "/" + my_filename)
                    
    return(my_df)


In [ ]:
API_call_to_df(indic_code="per_allsp.adq_pop_tot")

In [231]:
wer_indicator = API_call_to_df(indic_code="per_allsp.adq_pop_tot")

In [ ]:
wer_indicator2 = API_call_to_df(indic_code="per_allsp.adq_pop_tot")

In [189]:
wer_indicator.shape

(264, 25)

In [188]:
# Count the number of NaNs 
wer_indicator.isnull().sum()

Country Name        0
Country Code        0
Indicator Name      0
Indicator Code      0
2000              263
2001              263
2002              261
2003              261
2004              259
2005              251
2006              245
2007              244
2008              235
2009              232
2010              230
2011              230
2012              225
2013              240
2014              234
2015              231
2016              235
2017              245
2018              258
2019              263
2020              264
dtype: int64

In [156]:
wer_indicator.tail(5)

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2001,2002,2003,2004,2005,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 25
259,Kosovo,XKX,Adequacy of social protection and labor programs (% of total welfare of beneficiary households),per_allsp.adq_pop_tot,NaN,NaN,NaN,NaN,NaN,NaN,...,15.059514,17.811902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,"Yemen, Rep.",YEM,Adequacy of social protection and labor programs (% of total welfare of beneficiary households),per_allsp.adq_pop_tot,NaN,NaN,NaN,NaN,NaN,9.212461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,South Africa,ZAF,Adequacy of social protection and labor programs (% of total welfare of beneficiary households),per_allsp.adq_pop_tot,NaN,NaN,NaN,NaN,NaN,36.620761,...,NaN,NaN,34.201898,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,Zambia,ZMB,Adequacy of social protection and labor programs (% of total welfare of beneficiary households),per_allsp.adq_pop_tot,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.016747,NaN,NaN,NaN,NaN,NaN,NaN
263,Zimbabwe,ZWE,Adequacy of social protection and labor programs (% of total welfare of beneficiary households),per_allsp.adq_pop_tot,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,50.084065,NaN,22.702797,NaN,NaN


## Importing the Worldbank data catalog
We see that the catalog contain > 1500 series!!!

In [21]:
full_db_catalog = pd.read_excel(io=path + "/" + "WDI_CETS.xls",
             sheet_name = 1)

In [110]:
full_db_catalog

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
0,AG.AGR.TRAC.NO,"Agricultural machinery, tractors",Environment,Agricultural production,NaN,NaN
1,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer produc...,Environment,Agricultural production,NaN,NaN
2,AG.CON.FERT.ZS,Fertilizer consumption (kilograms per hectare ...,Environment,Agricultural production,NaN,NaN
3,AG.LND.AGRI.K2,Agricultural land (sq. km),Environment,Land use,NaN,NaN
4,AG.LND.AGRI.ZS,Agricultural land (% of land area),Environment,Land use,NaN,NaN
...,...,...,...,...,...,...
1499,WP15163_4.1,Mobile account (% age 15+),Financial Sector,Access,NaN,NaN
1500,WP15163_4.2,"Mobile account, male (% age 15+)",Financial Sector,Access,NaN,NaN
1501,WP15163_4.3,"Mobile account, female (% age 15+)",Financial Sector,Access,NaN,NaN
1502,WP15163_4.8,"Mobile account, income, poorest 40% (% ages 15+)",Financial Sector,Access,NaN,NaN


### Filtering the series that contain only %
We obtain only 755 variables which is a lot less!

In [57]:
raw_percentage_db_catalog = full_db_catalog[full_db_catalog['Series Name'].str.contains("%")]
raw_percentage_db_catalog

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
1,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer produc...,Environment,Agricultural production,NaN,NaN
4,AG.LND.AGRI.ZS,Agricultural land (% of land area),Environment,Land use,NaN,NaN
7,AG.LND.ARBL.ZS,Arable land (% of land area),Environment,Land use,NaN,NaN
9,AG.LND.CROP.ZS,Permanent cropland (% of land area),Environment,Land use,NaN,NaN
11,AG.LND.EL5M.RU.ZS,Rural land area where elevation is below 5 met...,Environment,Land use,NaN,NaN
...,...,...,...,...,...,...
1499,WP15163_4.1,Mobile account (% age 15+),Financial Sector,Access,NaN,NaN
1500,WP15163_4.2,"Mobile account, male (% age 15+)",Financial Sector,Access,NaN,NaN
1501,WP15163_4.3,"Mobile account, female (% age 15+)",Financial Sector,Access,NaN,NaN
1502,WP15163_4.8,"Mobile account, income, poorest 40% (% ages 15+)",Financial Sector,Access,NaN,NaN


In [129]:
pd.set_option('display.max_colwidth', None)

In [143]:
raw_percentage_db_catalog.head(10).sort_values(by = "Topic")#[raw_percentage_db_catalog[]]

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
29,BG.GSR.NFSV.GD.ZS,Trade in services (% of GDP),Economic Policy & Debt,Balance of payments,Current account,"Goods, services & income"
1,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer production),Environment,Agricultural production,NaN,NaN
4,AG.LND.AGRI.ZS,Agricultural land (% of land area),Environment,Land use,NaN,NaN
7,AG.LND.ARBL.ZS,Arable land (% of land area),Environment,Land use,NaN,NaN
9,AG.LND.CROP.ZS,Permanent cropland (% of land area),Environment,Land use,NaN,NaN
11,AG.LND.EL5M.RU.ZS,Rural land area where elevation is below 5 meters (% of total land area),Environment,Land use,NaN,NaN
13,AG.LND.EL5M.UR.ZS,Urban land area where elevation is below 5 meters (% of total land area),Environment,Land use,NaN,NaN
14,AG.LND.EL5M.ZS,Land area where elevation is below 5 meters (% of total land area),Environment,Land use,NaN,NaN
16,AG.LND.FRST.ZS,Forest area (% of land area),Environment,Land use,NaN,NaN
17,AG.LND.IRIG.AG.ZS,Agricultural irrigated land (% of total agricultural land),Environment,Land use,NaN,NaN


In [138]:
raw_percentage_db_catalog.columns.to_list()

['Series Code', 'Series Name', 'Topic', 'SubTopic1', 'SubTopic2', 'SubTopic3']

In [141]:
# Display the number of series per topic
raw_percentage_db_catalog.sort_values(by = "Topic").groupby(['Topic'], as_index = False).size()#[raw_percentage_db_catalog[]]

Topic
Economic Policy & Debt       105
Education                    124
Environment                   68
Financial Sector              35
Gender                         9
Health                       131
Infrastructure                 6
Poverty                       20
Private Sector & Trade        91
Public Sector                 27
Social Protection & Labor    139
dtype: int64

#### We Observe that the topics that contain the most number of series are:

1. **Social Protection & Labor**
2. **Health**
3. **Education**
4. **Economic Policy & Debt**

In [ ]:
# explore the various topics

In [148]:
raw_percentage_db_catalog[raw_percentage_db_catalog['Topic'] == "Social Protection & Labor"]

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
869,per_allsp.adq_pop_tot,Adequacy of social protection and labor programs (% of total welfare of beneficiary households),Social Protection & Labor,Performance,NaN,NaN
870,per_allsp.ben_q1_tot,Benefit incidence of social protection and labor programs to poorest quintile (% of total SPL benefits),Social Protection & Labor,Performance,NaN,NaN
871,per_allsp.cov_pop_tot,Coverage of social protection and labor programs (% of population),Social Protection & Labor,Performance,NaN,NaN
872,per_lm_alllm.adq_pop_tot,Adequacy of unemployment benefits and ALMP (% of total welfare of beneficiary households),Social Protection & Labor,Performance,NaN,NaN
873,per_lm_alllm.ben_q1_tot,Benefit incidence of unemployment benefits and ALMP to poorest quintile (% of total U/ALMP benefits),Social Protection & Labor,Performance,NaN,NaN
874,per_lm_alllm.cov_pop_tot,Coverage of unemployment benefits and ALMP (% of population),Social Protection & Labor,Performance,NaN,NaN
875,per_lm_alllm.cov_q1_tot,Coverage of unemployment benefits and ALMP in poorest quintile (% of population),Social Protection & Labor,Performance,NaN,NaN
876,per_lm_alllm.cov_q2_tot,Coverage of unemployment benefits and ALMP in 2nd quintile (% of population),Social Protection & Labor,Performance,NaN,NaN
877,per_lm_alllm.cov_q3_tot,Coverage of unemployment benefits and ALMP in 3rd quintile (% of population),Social Protection & Labor,Performance,NaN,NaN
878,per_lm_alllm.cov_q4_tot,Coverage of unemployment benefits and ALMP in 4th quintile (% of population),Social Protection & Labor,Performance,NaN,NaN
